<p style="text-align:right; font-size:14px;"> University of Studies of Florence
<p style="text-align:right; font-size:14px;"> Department of Engineering Information </p>
<p style="text-align:right; font-size:14px;"> Pistoia, April 1, 2021 </p>


<h1 align=center>Visual and Multimedia Recognition</h1>
<h2 align=center>
Unsupervised Learning with Deepcluster on VID Dataset
</h1>

<br>


In [1]:
__AUTHOR__ = {'lp': ("Lorenzo Pisaneschi",
                    "lorenzo.pisaneschi1@stud.unifi.it",
                     "https://github.com/pisalore/video_deepcluster")}

__TOPICS__ = ['Unsupervised Learning', 'Deep Learning', 'k-Means Clustering', 'AlexNet', 'Vid Dataset']


__KEYWORDS__ = ['Python', 'Pytorch', 'AI', 'Machine Learning', 'clustering']

<h1>Introduction</h1>

AI and Machine Learning algorithms are increasingly pervasive in daily life, with the result we need ever larger
dataset and ever more annotated data for our supervised learning algorithms. For this reason, unsupervised learning is
becoming important, in order to obtain useful information from input data to be maybe used later in a supervised learning
process.

<h3> Unsupervised Learning Challanges </h3>

Obviously, since our algorithm does not know how the output should be, we have to evaluate if the algorithm has learnt
something useful to our purpose. For example, using clustering algorithms, it is crucial to understand if data are
collected in a desired way for further inference on data.

<br>
<img style="display: block;
  margin-left: auto;
  margin-right: auto;
  width: 50%;" src="slides/images/img1.png" width="500">



<h3> Thrilling idea </h3>

The following points have to be fixed:

- Well tested pretrained CNNs are available, like those
obtained starting from famous ImageNet dataset, a fully supervised dataset consisting
in one million of images distributed over 1000 categories.

- Unsupervised learning algorithms can be applied on data from any domain, and unsupervised methods
could be applied to deep learning models.

- We'd like to have greater dataset to perform ever complex tasks with ever more data.

So, why not alternating clustering and convnet weights update using predicted cluster
assignments and build a classification model in order to annotate data without
any manual preprocessing?